In [6]:
# from llama_index import Vectorstore, SimpleDirectoryReader, ServiceContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [7]:
documents = SimpleDirectoryReader(r"./input").load_data()

In [8]:
documents

[Document(id_='ae4b87fd-8416-4d5d-b1fc-e11c6a45c3ee', embedding=None, metadata={'page_label': '1', 'file_name': 'circular_1691558721.pdf', 'file_path': 'c:\\Users\\user\\PycharmProjects\\RAG_Query PDFs\\input\\circular_1691558721.pdf', 'file_type': 'application/pdf', 'file_size': 270458, 'creation_date': '2024-06-03', 'last_modified_date': '2023-08-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='BANNARG HATTA MAIN ROAD, GOTTIGERE  P.O.  \nBANGALORE 83  \n \n CARMEL ACADEMY  \n \nCircular Regarding Second Instalment Fees Payment  \nPre-Primary and Grade I -XII \nDate: 09/08/2023  \n  \nDear Parent  \nGreetings from Carmel Academy   \n \n       This is a gentle reminder for the Second instalment fee. Payment should be made on \nor before 20th  August

In [9]:
system_prompt = """
You are a Q&A assistant. Your goal is to give response as accurately as possible based on the context provided.

"""
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
# You need to run this to get Hugging face token.
#huggingface-cli login

In [ ]:

import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [ ]:

service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
service_context

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:

index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("what is ai?")

In [ ]:
response